In [57]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from fuzzywuzzy import fuzz
from sklearn.cluster import AffinityPropagation
import difflib
# import geopandas as gpd

In [58]:
# # Loading OCPF data under all filing IDs
# ocpf_10176 = pd.read_csv('ocpf-output-10176.csv') # ocpf filings under filer 10176 Galvin, William F.

# ocpf_20176 = pd.read_csv('ocpf-output-20176.csv') # ocpf filings under filer 20176 Galvin, William F.

# ocpf_30176 = pd.read_csv('ocpf-output-30176.csv') # ocpf filings under filer 30176 Galvin, William F.

# total_ocpf = pd.concat([ocpf_10176, ocpf_20176,ocpf_30176], ignore_index=True) # concatenating to one dataframe

In [59]:
tax_credits = pd.read_csv('round_53_17_latest_round.csv')
tax_credits = tax_credits.rename(columns={'Name of Property': 'property_name',
                            'Prior Rounds Applications': 'prior_rounds_applications',
                            'Proponent Name': 'proponent_name',
                            'Remaining Credit to Award': 'remaining_credit_to_award',
                            'Total Awards': 'total_awards',
                            'Part 2 Qualified Rehab. $':'part_2_qualified_rehab_$',
                            'Part 2 Total Project Cost': 'part_2_total_project_cost'})
# tax_credits.head()

In [60]:
# https://www.analyticsinsight.net/company-names-standardization-using-a-fuzzy-nlp-approach/ 
def clean_special_characters(txt):
    seps = [" ", ";", ":", ".","*","#","@","|","/","\\","-","_","?","%","!","^","(",")"]
    default_sep = seps[0]

    for sep in seps[1:]:
        txt = txt.replace(sep, default_sep)
    re.sub(' +',' ',txt)
    temp_list = [i.strip() for i in txt.split(default_sep)]
    temp_list = [i for i in temp_list if i]
    return " ".join(temp_list)

def clean_stopword(txt):
    temp_list = txt.split(" ")
    entity_suffix = ['corp','inc','ltd','llc','lc','lp'] # https://www.harborcompliance.com/information/company-suffixes
    temp_list = [i for i in temp_list if i not in stopwords.words('english') and i not in entity_suffix]
    return " ".join(temp_list)

def data_cleaning(data, nameCol = 'Entity', dropForeign=True):
    data.dropna(subset=nameCol, inplace = True)
    data = data.rename_axis('CompanyID').reset_index()
    data['nonAscii_count'] = data[nameCol].apply(lambda x: sum([not c.isascii() for c in x]))
    if dropForeign:
        data=data[data.nonAscii_count==0]
    else:
        pass
    data.drop('nonAscii_count', axis=1, inplace=True)
    data_clean=data.copy()
    data_clean['EntityName_clean'] = data_clean[nameCol].apply(lambda x: x.lower())
    data_clean['EntityName_clean'] = data_clean['EntityName_clean'].apply(clean_special_characters)
    data_clean['EntityName_clean'] = data_clean['EntityName_clean'].apply(clean_stopword)
    return data_clean

In [61]:
def fuzz_similarity(cust_names):
    similarity_array = np.ones((len(cust_names), (len(cust_names)))) * 100
    for i in range(1,len(cust_names)):
        for j in range(i):
            s1 = fuzz.token_set_ratio(cust_names[i],cust_names[j]) + 0.000000000001
            s2 = fuzz.partial_ratio(cust_names[i],cust_names[j]) + 0.000000000001
            similarity_array[i][j] = 2*s1*s2 / (s1+s2)
    for i in range(len(cust_names)):
        for j in range(i+1,len(cust_names)):
            similarity_array[i][j] = similarity_array[j][i]
    np.fill_diagonal(similarity_array,100)
    return similarity_array

def company_clusters(data, nameCol='Entity', dropForeign=True):
    data_clean = data_cleaning(data, nameCol=nameCol, dropForeign=dropForeign)
    cust_names = data_clean.EntityName_clean.to_list()
    cust_ids = data_clean.CompanyID.to_list()

    similarity_array = fuzz_similarity(cust_names)
    clusters = AffinityPropagation(affinity='precomputed').fit_predict(similarity_array)
    df_clusters=pd.DataFrame(list(zip(cust_ids, clusters)), columns = ['CompanyID','cluster'])

    df_eval = df_clusters.merge(data_clean,on='CompanyID',how='left')
    return df_eval

In [62]:
def standard_name(df_eval):
    d_standard_name = {}
    for cluster in df_eval.cluster.unique():
        names = df_eval[df_eval['cluster']==cluster].EntityName_clean.to_list()
        l_common_substring = []
        if len(names)>1:
            for i in range(0,len(names)):
                for j in range(i+1,len(names)):
                    seqMatch = difflib.SequenceMatcher(None, names[i],names[j])
                    match = seqMatch.find_longest_match(0,len(names[i]),0,len(names[j]))
                    if (match.size!=0):
                        l_common_substring.append(names[i][match.a: match.a + match.size].strip())
            n = len(l_common_substring)
            counts = Counter(l_common_substring)
            get_mode = dict(counts)
            mode = [k for k, v in get_mode.items() if v == max(list(counts.values()))]
            d_standard_name[cluster] = ";".join(mode)
        else:
            d_standard_name[cluster] = names[0]
    df_standard_names = pd.DataFrame(list(d_standard_name.items()), columns = ['cluster','StandardName'])
    df_eval = df_eval.merge(df_standard_names, on ='cluster',how='left')
    df_eval['Score_with_standard'] = df_eval.apply(lambda x: fuzz.token_set_ratio(x['StandardName'],x['EntityName_clean']), axis= 1)
    df_eval['standard_name_withoutSpaces'] = df_eval.StandardName.apply(lambda x: x.replace(" ",""))
    for name in df_eval.standard_name_withoutSpaces.unique():
        if len(df_eval[df_eval.standard_name_withoutSpaces == name].cluster.unique()) > 1:
            df_eval.loc[df_eval.standard_name_withoutSpaces==name, 'StandardName'] = name
    return df_eval.drop('standard_name_withoutSpaces', axis=1)

In [63]:
# step_1 = data_cleaning(tax_credits)
step_2 = company_clusters(tax_credits)
step_2

/Users/hoperuse/Projects/htc-analysis/new_env/lib/python3.9/site-packages/sklearn/cluster/_affinity_propagation.py:236: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(


,CompanyID,cluster,Unnamed: 0,City/Town,property_name,Use**,prior_rounds_applications,proponent_name,remaining_credit_to_award,total_awards,20% QRE,Qualified Rehab. $,Total Project Cost,HRC #,Entity,part_2_qualified_rehab_$,part_2_total_project_cost,Count_prior_rounds_applied,EntityName_clean
0,0,0,2431,Lawrence,Wood Worsted Mill Building C (formerly Office ...,R,"13 4, 15, 16,17, 18, 19, 20,21, 22, 23, 24,...",Salvatore Lupoli,574607.60,3000000,3574607.60,17873038.0,22481348.0,117.0,200 Riverwalk LLC,NaN,NaN,21.0,200 riverwalk
1,1,87,805,Lowell,Hamilton Canal Lofts (Phase 2 of 2),"R, C","6,7,8,9,10,11,12,13,14,15, 16,17, 18, 19, 20,2...",Sean McDonnell,585826.40,12800000,7485826.40,37429132.0,45006500.0,54.0,"HCL Acquisition LLC, subsidiary of Architectur...",NaN,NaN,21.0,"hcl acquisition llc, subsidiary architectural ..."
2,2,70,1234,Waltham,Waltham Watch Company Phased (Phases 10-11 of 12),"R, C","11, 12, 13, 14,15, 16, 17, 18,19, 20, 21, 22,2...",Ian McGill,4259000.00,4700000,22729452.80,113647264.0,132824000.0,96.0,"Watch City Ventures LLC, c/o Berkeley Investme...",NaN,NaN,20.0,"watch city ventures llc, c berkeley investments,"
3,3,11,1350,Boston / Charlestown,"Terminal Storage Company, 267 Medford Street",R,"12,13, 15, 16,17, 18, 19, 21,22, 23, 24, 25,26...",Michael Rauseo,1740000.00,6700000,5640000.00,28200000.0,37200000.0,103.0,Suffolk Medford LLC,NaN,NaN,19.0,suffolk medford
4,4,92,1926,Fitchburg,"Fitchburg Yarn Mill, 1428 Main Street",R,"21, 22, 23, 24,25, 26, 27, 28,29, 30, 31, 32,3...",Lawrence Curtis,755994.80,4900000,5655994.80,28279974.0,37440936.0,275.0,Fitchburg Yarn Limited Partnership,NaN,NaN,18.0,fitchburg yarn limited partnership
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,462,30,1743,Natick,13 West Central Street,"R, C",36,Dean Calivas,870434.60,0,870434.60,4352173.0,7459195.0,677.0,Stonegate West Central LLC,NaN,NaN,NaN,stonegate west central
457,464,16,1967,Springfield,"New Court Terrace: FrederickB. Taylor Block, 7...",R,38,Peter Gagliardi,166766.00,0,166766.00,833830.0,1104750.0,742.0,"HAP, Inc.",NaN,NaN,NaN,"hap,"
458,465,16,1968,Springfield,"New Court Terrace: William Birnie Block, 68-70...",R,38,Peter Gagliardi,157086.40,0,157086.40,785432.0,1177776.0,743.0,"HAP, Inc.",NaN,NaN,NaN,"hap,"
459,466,7,2193,Haverhill,"St. Gregory School, 108 Harrison Street",R,40,William F. Lipchitz,1013757.40,0,1013757.40,5068787.0,8638007.0,788.0,Common Ground Development Corporation,NaN,NaN,NaN,common ground development corporation


In [64]:
from collections import Counter
step_3 = standard_name(step_2)
step_3

,CompanyID,cluster,Unnamed: 0,City/Town,property_name,Use**,prior_rounds_applications,proponent_name,remaining_credit_to_award,total_awards,...,Qualified Rehab. $,Total Project Cost,HRC #,Entity,part_2_qualified_rehab_$,part_2_total_project_cost,Count_prior_rounds_applied,EntityName_clean,StandardName,Score_with_standard
0,0,0,2431,Lawrence,Wood Worsted Mill Building C (formerly Office ...,R,"13 4, 15, 16,17, 18, 19, 20,21, 22, 23, 24,...",Salvatore Lupoli,574607.60,3000000,...,17873038.0,22481348.0,117.0,200 Riverwalk LLC,NaN,NaN,21.0,200 riverwalk,river,56
1,1,87,805,Lowell,Hamilton Canal Lofts (Phase 2 of 2),"R, C","6,7,8,9,10,11,12,13,14,15, 16,17, 18, 19, 20,2...",Sean McDonnell,585826.40,12800000,...,37429132.0,45006500.0,54.0,"HCL Acquisition LLC, subsidiary of Architectur...",NaN,NaN,21.0,"hcl acquisition llc, subsidiary architectural ...",tion,12
2,2,70,1234,Waltham,Waltham Watch Company Phased (Phases 10-11 of 12),"R, C","11, 12, 13, 14,15, 16, 17, 18,19, 20, 21, 22,2...",Ian McGill,4259000.00,4700000,...,113647264.0,132824000.0,96.0,"Watch City Ventures LLC, c/o Berkeley Investme...",NaN,NaN,20.0,"watch city ventures llc, c berkeley investments,",investments,100
3,3,11,1350,Boston / Charlestown,"Terminal Storage Company, 267 Medford Street",R,"12,13, 15, 16,17, 18, 19, 21,22, 23, 24, 25,26...",Michael Rauseo,1740000.00,6700000,...,28200000.0,37200000.0,103.0,Suffolk Medford LLC,NaN,NaN,19.0,suffolk medford,urban edge,32
4,4,92,1926,Fitchburg,"Fitchburg Yarn Mill, 1428 Main Street",R,"21, 22, 23, 24,25, 26, 27, 28,29, 30, 31, 32,3...",Lawrence Curtis,755994.80,4900000,...,28279974.0,37440936.0,275.0,Fitchburg Yarn Limited Partnership,NaN,NaN,18.0,fitchburg yarn limited partnership,limitedpartnership,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,462,30,1743,Natick,13 West Central Street,"R, C",36,Dean Calivas,870434.60,0,...,4352173.0,7459195.0,677.0,Stonegate West Central LLC,NaN,NaN,NaN,stonegate west central,west newton rutland,49
457,464,16,1967,Springfield,"New Court Terrace: FrederickB. Taylor Block, 7...",R,38,Peter Gagliardi,166766.00,0,...,833830.0,1104750.0,742.0,"HAP, Inc.",NaN,NaN,NaN,"hap,",housing,20
458,465,16,1968,Springfield,"New Court Terrace: William Birnie Block, 68-70...",R,38,Peter Gagliardi,157086.40,0,...,785432.0,1177776.0,743.0,"HAP, Inc.",NaN,NaN,NaN,"hap,",housing,20
459,466,7,2193,Haverhill,"St. Gregory School, 108 Harrison Street",R,40,William F. Lipchitz,1013757.40,0,...,5068787.0,8638007.0,788.0,Common Ground Development Corporation,NaN,NaN,NaN,common ground development corporation,development corporation,100


In [65]:
step_3.sort_values(by='Score_with_standard',ascending=False)

,CompanyID,cluster,Unnamed: 0,City/Town,property_name,Use**,prior_rounds_applications,proponent_name,remaining_credit_to_award,total_awards,...,Qualified Rehab. $,Total Project Cost,HRC #,Entity,part_2_qualified_rehab_$,part_2_total_project_cost,Count_prior_rounds_applied,EntityName_clean,StandardName,Score_with_standard
230,230,30,2152,Boston / South End,West Newton Street and Rutland Street Apartmen...,R,"37, 38, 39, 40",Vanessa Calderón- Rosado,140449.60,150000,...,1452248.0,2597106.0,688.0,West Newton Rutland LLC,NaN,NaN,4.0,west newton rutland,west newton rutland,100
251,252,34,1187,Cambridge,"Hotel Austin, 70 Bishop Allen Drive",R,"28, 29, 30",Robert MacArthur,146178.20,250000,...,1980891.0,3158275.0,436.0,Bishop Allen Apartments LLC c/o Just- A-Start ...,NaN,NaN,3.0,bishop allen apartments c start corporation,bishop allen apartments c start corporation,100
249,249,33,962,Boston / North End,"Boston Printing Department Building, 130-140 R...",A,"26, 27, 28",Miguel Gomez- Ibanez,800652.20,1300000,...,10503261.0,14982105.0,386.0,North Bennet Street School,NaN,NaN,3.0,north bennet street school,school,100
248,248,28,1400,Nantucket,Nesbitt Inn (the former William T. Swain House...,H,"30, 31, 32",Jay Bisognano,529828.80,300000,...,4149144.0,8409256.0,514.0,Mount Vernon Company,NaN,NaN,3.0,mount vernon company,company,100
247,247,11,1368,Boston / Roxbury,1865 Columbus Avenue,R,"30, 31, 32",Sharon Zimmerman,305202.40,200000,...,2526012.0,5349312.0,509.0,Dimock-Bragdon Limited Partnership c/o Urban Edge,NaN,NaN,3.0,dimock bragdon limited partnership c urban edge,urban edge,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,346,32,2069,New Bedford,141 Union Street,"R, C","35, 36",Teri Bernert,89561.00,150000,...,1197805.0,1463205.0,638.0,Waterfront Historic Area League,NaN,NaN,2.0,waterfront historic area league,wa,12
153,153,86,1096,Cambridge,"Craigie Arms, 122 Mount Auburn Street",R,"25, 26, 27, 28,29",Jane Carbone,674691.40,700000,...,6873457.0,23369608.0,364.0,"Executive Director, Chapman Arms, LLC c/o Home...",NaN,NaN,5.0,"executive director, chapman arms, c homeowner'...",ner,11
460,467,18,2477,Worcester,"MJ Whittall, Mill Building #2, 6 Brussels Street","C, O",42,Lisa Dileo,121285.32,0,...,606426.6,671901.6,836.0,MJ Whittall LLC,NaN,NaN,NaN,mj whittall,development,9
108,108,87,1164,Boston / Back Bay,"Boston Consolidated Gas Company, 100 Arlington...","R, C","24, 25, 26, 27,28, 29, 30",Dean F. Stratouly,13451209.40,3000000,...,79756047.0,112627605.0,330.0,"100 Arlington Acquisition Company, LLC; Delewa...",NaN,NaN,7.0,"100 arlington acquisition company, deleware li...",tion,8


In [66]:
step_3.loc[step_3.cluster == 0]

,CompanyID,cluster,Unnamed: 0,City/Town,property_name,Use**,prior_rounds_applications,proponent_name,remaining_credit_to_award,total_awards,...,Qualified Rehab. $,Total Project Cost,HRC #,Entity,part_2_qualified_rehab_$,part_2_total_project_cost,Count_prior_rounds_applied,EntityName_clean,StandardName,Score_with_standard
0,0,0,2431,Lawrence,Wood Worsted Mill Building C (formerly Office ...,R,"13 4, 15, 16,17, 18, 19, 20,21, 22, 23, 24,...",Salvatore Lupoli,574607.6,3000000,...,17873038.0,22481348.0,117.0,200 Riverwalk LLC,NaN,NaN,21.0,200 riverwalk,river,56
10,10,0,2316,Lawrence,"Wood Worsted Mill Building F, 280 Merrimack St...",C,"27, 28, 29, 30,31, 32, 33, 34,35, 36, 37, 38,3...",Salvatore Lupoli,5493282.0,5000000,...,52466410.0,63324227.0,412.0,280 Riverwalk LLC,NaN,NaN,15.0,280 riverwalk,river,56
37,37,0,334,New Bedford,Whitman Mill Number 2,R,"13,14,15, 16, 17,18, 19, 20, 21",Steve Ricciardi,225010.6,3500000,...,15625053.0,20702873.0,128.0,Victoria Riverview LLC,NaN,NaN,9.0,victoria riverview,river,43
